# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [2]:
import json

def organ_failure_history(organ_failure, config_file):
    """ int, str -> int
    This function takes any type of history of severe organ failure and
    gives back an output associated with the medical criteria. Examples are shown below...
    
    
    >>>organ_failure_history(["Elective"],ORGAN_CONFIG_FILE )
    2
    
    
    >>>organ_failure_history(["Emergency"],ORGAN_CONFIG_FILE )
    5
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    organ_failure_score = config.get("Organ Failure History")
    
    for organ in organ_failure:
        if organ in organ_failure_score:
            score = organ_failure_score.get(organ)
    
    ### END SOLUTION

    return score

In [3]:
config_file = "apache.json"

In [4]:
organ_failure_history(["Emergency"], config_file)

5

In [5]:
config = json.load(open(config_file))

In [6]:
organ_failure_score = config.get("Organ Failure History")

In [7]:
type(organ_failure_score)

dict

In [8]:
organ_failure_history(["None"], config_file)

0

In [9]:
import json
def score(val, config_file, range_name):
    """ int, str, str -> int
    This function takes any value in the json file and 
    gives back an output associated with the medical criteria. Examples are shown below...
    
    
    >>>score(38, config_file, "Respiratory Rate")
    3
    
    
    >>>score(120, config_file, "Heart Rate")
    2
    """
        
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    rules = config.get(range_name)
    
    for rule in rules:
        if val >= rule.get('min') and val < rule.get('max'):
            score = rule.get('points')
        
    
    ### END SOLUTION

    return score

In [10]:
score(80, config_file, "Age")

6

In [11]:
import json

def PaO2_score(FiO2, PaO2, config_file):
    """ int, int, str -> int
    This function takes the FiO2 levels and PaO2 levels (if FiO2<50%) 
    and outputs a score value. Examples are shown below...
    
    >>>(PaO2_score(36, 60, config_file)
    3
    
    
    >>>(PaO2_score(49, 71, config_file)
    0
    """
    
    score = 0  
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    FiO2_rules = config.get("FiO2")
    PaO2_rules = {}
    
    try:
        for rule in FiO2_rules:
            if FiO2 >= rule.get('min') and FiO2 < rule.get('max'):
                PaO2_rules = rule.get("PaO2")

        for PaO2_rule in PaO2_rules:
            if PaO2 >= PaO2_rule.get('min') and PaO2 < PaO2_rule.get('max'):
                score = PaO2_rule.get('points')
            
    except:
        score= 0
            
    
    ### END SOLUTION

    return score

    

In [12]:
import json

def Aa_Gradient_score(FiO2, Aa_Gradient, config_file):
    """ int, int, str -> int
    This function takes the FiO2 levels and Aa_Gradient levels (if FiO2>50%) 
    and outputs a score value. Examples are shown below...
    >>>(Aa_Gradient_score(54, 600, config_file)
    4
    
    >>>(Aa_Gradient_score(53, 370, config_file)
    3
    """
    
    score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    FiO2_rules = config.get("FiO2")
    Aa_Gradient_rules = {}
    
    try:
        for rule in FiO2_rules:
            if FiO2 >= rule.get('min') and FiO2 < rule.get('max'):
                Aa_Gradient_rules = rule.get("A-a gradient")

        for rule in Aa_Gradient_rules:
            if Aa_Gradient >= rule.get('min') and Aa_Gradient < rule.get('max'):
                score = rule.get('points')
    except: 
        score = 0
            
    
    
    ### END SOLUTION

    return score

In [13]:
config_file = "apache.json"

In [14]:
config = json.load(open(config_file))

In [15]:
FiO2_rules = config.get("FiO2")

In [16]:
import json

def acute_renal_score(Creatinine, renal_failure, config_file):
    """ int, string, string -> int
    This function takes the creatinine levels and acute renal failure and outputs a score value. 
    Examples are below...
    >>>acute_renal_score(1.7, "Acute Renal Failure", config_file)
    4
    >>>acute_renal_score(2.5, "Acute Renal Failure", config_file)
    6
    """
    
    A_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    Creatinine_rules = config.get("Creatinine")
    
    
    try:
        for rule in Creatinine_rules:
            Acute_Renal_Failure= Creatinine_rules.get("Acute Renal Failure")
        
        for rule in Acute_Renal_Failure:
            if Creatinine >= rule.get('min') and Creatinine < rule.get('max') and renal_failure== "Acute Renal Failure":
                A_score = rule.get('points')
                
    except:
        A_score= 0
            
    
    ### END SOLUTION

    return A_score

In [17]:
import json

def chronic_renal_score(Creatinine, renal_failure, config_file):
    """ int, string, string -> int
    This function takes the creatinine levels and chronic renal failure and outputs a score value. 
    Examples are below...
    >>>chronic_renal_score(4, "Chronic Renal Failure" config_file)
    4
    >>>chronic_renal_score(2.7, "Chronic Renal Failure" config_file)
    3
    """
    
    C_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    Creatinine_rules = config.get("Creatinine")
    
    
    try:
        for rule in Creatinine_rules:
            Chronic_Renal_Failure= Creatinine_rules.get("Chronic Renal Failure")
        
        for rule in Chronic_Renal_Failure:
            if Creatinine >= rule.get('min') and Creatinine < rule.get('max') and renal_failure== "Chronic Renal Failure":
                C_score = rule.get('points')
                
        
            
                

        
            
    except:
        C_score= 0
            
    
    ### END SOLUTION

    return C_score

In [18]:
import json

def creatinine_score(Creatinine, config_file):
    """ int,string -> int
    This function takes the creatinine levels and outputs a score value. Examples are below...
    >>>(creatinine_score(1.3, config_file)
    0
    >>>creatinine_score(0.3, config_file)
    2
    """
    
    N_score = 0
    
    ### BEGIN SOLUTION
    
    config = json.load(open(config_file))
    Creatinine_rules = config.get("Creatinine")
    
    
    try:
        for rule in Creatinine_rules:
            No_Failure_rules= Creatinine_rules.get("No Failure")
        
        for rule in No_Failure_rules:
            if Creatinine >= rule.get('min') and Creatinine < rule.get('max'):
                N_score = rule.get('points')
                
        
            
                

        
            
    except:
        N_score= 0
            
    
    ### END SOLUTION

    return N_score

In [19]:
ORGAN_CONFIG_FILE= 'apache.json'

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you have a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [20]:
config_file= "apache.json"

In [21]:
assert(organ_failure_history(["Nonoperative"],ORGAN_CONFIG_FILE ) == 5)
assert(organ_failure_history([],ORGAN_CONFIG_FILE ) == 0)
assert(organ_failure_history(["Emergency"],ORGAN_CONFIG_FILE ) == 5)
assert(organ_failure_history(["Elective"],ORGAN_CONFIG_FILE ) == 2)

In [22]:
assert(score(48, config_file, "Age")== 2)
assert(score(38.7, config_file, "Temperature")== 1)
assert(score(120, config_file, "Heart Rate")== 2)
assert(score(38, config_file, "Respiratory Rate")== 3)

In [23]:
assert(PaO2_score(49, 57, config_file)==3)
assert(PaO2_score(48, 54, config_file)==4)
assert(PaO2_score(47, 62, config_file)==1)
assert(PaO2_score(51, 62, config_file)==0)
assert(PaO2_score(49, 71, config_file)==0)
assert(PaO2_score(36, 60, config_file)==3)

In [24]:
assert(Aa_Gradient_score(51, 200, config_file)==2)
assert(Aa_Gradient_score(49, 200, config_file)==0)
assert(Aa_Gradient_score(53, 370, config_file)==3)
assert(Aa_Gradient_score(54, 600, config_file)==4)

In [25]:
assert(acute_renal_score(4, "Acute Renal Failure", config_file)==8)
assert(acute_renal_score(2.5, "Acute Renal Failure", config_file)==6)
assert(acute_renal_score(1.7, "Acute Renal Failure", config_file)==4)

In [26]:
assert(chronic_renal_score(4, "Chronic Renal Failure" , config_file)==4)
assert(chronic_renal_score(2.7, "Chronic Renal Failure" , config_file)==3)
assert(chronic_renal_score(1.8, "Chronic Renal Failure" , config_file)==2)

In [27]:
assert(creatinine_score(1.3, config_file)==0)
assert(creatinine_score(0.3, config_file)==2)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [28]:
def apache_score(organ_failure, Age, Temp, pH, HR, RR, Na, K, Creatinine, renal_failure, Hema, WBC, Glasgow, FiO2, PaO2, Aa_Gradient):
    ap_score=0
    """
    This function takes all the values from the Apache II medical criteria and gives back an output.
    Examples are shown below...
    
    >>>>(apache_score(["Elective"], 3, 4, 7.2, 74, 53, 106, 3.2, 1.2, "Acute Renal Failure", 60, 22, 11, 66, 59, 482)
    31
    
    >>>>(apache_score(["Nonoperative"], 62, 28, 7.1, 56, 7, 177, 0.4, 1.6, "Acute Renal Failure", 38, 50, 7, 57, 74, 527)
    47
    """
    config_file = "apache.json"
    ap_score += organ_failure_history(organ_failure, config_file)
    ap_score += score(Age, config_file,"Age")
    ap_score += score(Temp, config_file,"Temperature")
    ap_score += score(pH, config_file,"pH")
    ap_score += score(HR, config_file,"Heart Rate")
    ap_score += score(RR, config_file,"Respiratory Rate")
    ap_score += score(Na, config_file,"Sodium")
    ap_score += score(K, config_file,"Potassium")
    ap_score += creatinine_score(Creatinine, config_file)
    ap_score += acute_renal_score(Creatinine, renal_failure, config_file)
    ap_score += chronic_renal_score(Creatinine, renal_failure, config_file)
    ap_score += score(Hema, config_file,"Hematocrit")
    ap_score += score(WBC, config_file,"White Blood Count")
    ap_score += 15- Glasgow
    ap_score += Aa_Gradient_score(FiO2, Aa_Gradient, config_file)
    ap_score += PaO2_score(FiO2, PaO2, config_file) 
    return ap_score

In [29]:
apache_score(["None"], 50, 32, 7.7, 10, 26, 144, 6.2, 1.8, "Chronic Renal Failure", 56, 50, 9, 41, 61, 217)

31

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases to run through, but you should do some of own your before moving on.

In [30]:
assert(apache_score(["Elective"], 3, 4, 7.2, 74, 53, 106, 3.2, 1.2, "Acute Renal Failure", 60, 22, 11, 66, 59, 482)==31)
assert(apache_score(["Nonoperative"], 62, 28, 7.1, 56, 7, 177, 0.4, 1.6, "Acute Renal Failure", 38, 50, 7, 57, 74, 527)==47)
assert(apache_score(["Nonoperative"], 10, 32, 7.7, 10, 26, 144, 6.2, 1.8, "Chronic Renal Failure", 56, 50, 9, 41, 61, 217)==34)

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [31]:
import requests
import csv
url= 'https://hds5210-2020.s3.amazonaws.com/TestPatients.csv'
with requests.Session() as s:
    download= s.get(url)
    
decoded_content= download.content.decode('utf-8')
    
cr=csv.reader(decoded_content.splitlines(), delimiter= ',')
patients= list(cr)

In [ ]:
patients

In [ ]:
for patient in patients:
    scores=[apache_score(patient[0], int(patient[1]), int(patient[2]), float(pa), int(HR[val]), int(RR[val]), int(Na[val]), float(K[val]), float(Cr[val]), rf[val], int(He[val]), int(WBC[val]), int(Gl[val]), int(Fi[val]), int(Pa[val]), int(Aa[val]))
    
    

In [32]:
ID=[]
organ=[]
Age=[]
pH=[]
temp=[]
HR=[]
RR=[]
Na=[]
K=[]
Cr=[]
rf=[]
He=[]
WBC=[]
Gl=[]
Fi=[]
Pa=[]
Aa=[]
for patient in patients:
    ID.append(patient[0])
    organ.append(patient[1])
    Age.append(patient[2])
    temp.append(patient[3])
    pH.append(patient[4])
    HR.append(patient[5])
    RR.append(patient[6])
    Na.append(patient[7])
    K.append(patient[8])
    Cr.append(patient[9])
    rf.append(patient[10])
    He.append(patient[11])
    WBC.append(patient[12])
    Gl.append(patient[13])
    Fi.append(patient[14])
    Pa.append(patient[15])
    Aa.append(patient[16])

In [34]:
for patient in patients:
    patient[1]

In [ ]:
apache_2_scores=[]
val=1
length=335
while val<length:
    scores=[apache_score([organ[val]], int(Age[val]), int(temp[val]), float(pH[val]), int(HR[val]), int(RR[val]), int(Na[val]), float(K[val]), float(Cr[val]), rf[val], int(He[val]), int(WBC[val]), int(Gl[val]), int(Fi[val]), int(Pa[val]), int(Aa[val]))]
    val+= 1
    apache_2_scores.append(scores)


### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [ ]:
apache_2_scores